# 일반적인 연립선형방정식

벡터 공간 $\mathbb{R}^n$을 살펴보기 전에 연립선형방정식(이하 선형 시스템)의 일반적인 질문에 대한 답변을 할 준비가 되어있습니다. 모든 선형 시스템의 경우 다음 명제 중 정확히 한 가지가 참입니다:

1. 시스템에 해가 존재하지 않는다(inconsistent).
2. 시스템에 하나의 해가 존재한다.
3. 시스템에 해가 무한하다.

주어진 선형 시스템에 앞선 세 가지 명제 중 어떤 것이 참인지 확인하는 것이 핵심 사항 입니다. 이전 장에서는 고유한 해가 있는 선형 시스템에 중점을 두었습니다.

이제 다른 가능성에 대해 자세히 살펴보겠습니다. 그리고 방정식과 미지수가 같은 선형 시스템이 아니라 $m$ 개의 방정식과 $n$ 개의 미지수를 가진 선형 시스템, 즉 일반적인 경우를 고려하겠습니다. 

기약 행 사다리꼴(이하, RREF)을 주로 사용 할 예정이며, RREF의 유용성에 대해서 확인할 수 있을 것으로 기대합니다.

In [6]:
import numpy as np

def row_swap(A,k,l):
    m = A.shape[0]  # m is number of rows in A
    n = A.shape[1]  # n is number of columns in A
    
    B = np.copy(A).astype('float64')

    for j in range(n):
        temp = B[k][j]
        B[k][j] = B[l][j]
        B[l][j] = temp
        
    return B

def row_scale(A,k,scale):
    m = A.shape[0]  # m is number of rows in A
    n = A.shape[1]  # n is number of columns in A
    
    B = np.copy(A).astype('float64')

    for j in range(n):
        B[k][j] *= scale
        
    return B

def row_add(A,k,l,scale):
    m = A.shape[0]  # m is number of rows in A
    n = A.shape[1]  # n is number of columns in A
    
    B = np.copy(A).astype('float64')
        
    for j in range(n):
        B[l][j] += B[k][j]*scale
        
    return B

def full_row_reduction(A, tol = 1e-14):
    m = A.shape[0]  # m is number of rows in A
    n = A.shape[1]  # n is number of columns in A

    B = np.copy(A).astype('float64')

    # Set initial pivot search position
    pivot_row = 0
    pivot_col = 0
    
    # Continue steps of elimination while possible pivot positions are 
    # within bounds of the array.
    
    while(pivot_row < m and pivot_col < n):

        # Set pivot value to current pivot position
        pivot = B[pivot_row,pivot_col]
        
        # If pivot is zero, search down current column, and then subsequent
        # columns (at or beyond pivot_row) for the next nonzero entry in the 
        # array is found, or the last entry is reached.

        row_search = pivot_row
        col_search = pivot_col
        search_end = False

        while(pivot == 0 and not search_end):
            if(row_search < m-1):
                row_search += 1
                pivot = B[row_search,col_search]
            else:
                if(col_search < n-1):
                    row_search = pivot_row
                    col_search += 1
                    pivot = B[row_search,col_search]
                else:  
                    # col_search = n-1 and row_search = m-1
                    search_end = True
                        
        # Swap row if needed to bring pivot to position for rref
        if (pivot != 0 and pivot_row != row_search):
            B = row_swap(B,pivot_row,row_search)
            pivot_row, row_search = row_search, pivot_row
            
        # Set pivot position to search position
        pivot_row = row_search
        pivot_col = col_search
            
        # If pivot is nonzero, carry on with elimination in pivot column 
        if (pivot != 0):
            
            # Set pivot entry to one
            B = row_scale(B,pivot_row,1./B[pivot_row,pivot_col])

            # Create zeros above pivot
            for i in range(pivot_row):    
                B = row_add(B,pivot_row,i,-B[i][pivot_col])
                # Force known zeros
                B[i,pivot_col] = 0

            # Create zeros below pivot
            for i in range(pivot_row+1,m):    
                B = row_add(B,pivot_row,i,-B[i][pivot_col])
                # Force known zeros
                B[i,pivot_col] = 0

            # Force small numbers to zero to account for roundoff error
            for i in range(m):
                for j in range(n):
                    if abs(B[i,j])< tol :
                        B[i,j] = 0

        # Advance to next possible pivot position
        pivot_row += 1
        pivot_col += 1
        
    return B


## 기약 행 사다리꼴

앞선 장에서 선형 시스템의 해를 구하는 방법 중 역행렬, LU 분해 등과 같은 방법을 사용해서 선형 연립방정식을 해결하였습니다. 일반적인 선형 시스템의 경우 기본 형 연산을 사용하여 RREF 형태로 만드는 것이 일반적입니다.

1. 각 행의 첫 번째 $0$이 아닌 항목은 $1$입니다. 이러한 항목을 **피벗(pivots)**이라고 합니다. 
1. 각 피벗은 윗 행의 피벗 오른쪽에 위치합니다. 
1. 각 피벗 위와 아래의 항목은 0입니다. 
1. 모든 0인 행은 다른 행 아래에 위치합니다.

다음은 RREF로 구성된 간단한 예제입니다.
   
$$
\begin{matrix}
\left[ \begin{array}{cccc} 1 & 0 & 0 & * \\ 0 & 1 & 0 & * \\ 0 & 0 & 1 & * \end{array}\right]
\end{matrix}
$$
   
$$
\begin{matrix}
\left[ \begin{array}{ccccc} 1 & 0 & * & 0 & *  \\ 0 & 1 & * & 0 & * 
\\ 0 & 0 & 0 & 1 & * \\ 0 & 0 & 0 & 0 & 0 \end{array}\right]
\end{matrix}
$$

$$
\begin{matrix}
\left[ \begin{array}{cccccc} 1 & * & 0 & 0 & * & * \\ 0 & 0 & 1 & 0 & * & * \\ 0 & 0 & 0 & 1 & * & * \end{array}\right]
\end{matrix}
$$

In [7]:
import numpy as np
import sympy as sp

C = np.random.randint(-2,3,size=(3,5))
C_reduced = full_row_reduction(C)

print(C)
print('\n')
print(C_reduced)

[[-1  0 -1  1  1]
 [-2 -1  1 -2  1]
 [-2 -1  0 -2  1]]


[[ 1.  0.  0. -1. -1.]
 [ 0.  1.  0.  4.  1.]
 [ 0.  0.  1.  0.  0.]]


## 피벗 위치 및 자유 변수

먼저 RREF를 사용하여 **자유 변수(free variables)**가 있는 선형 시스템을 푸는데 어떻게 도움이 되는지 살펴봅니다. 자유 변수는 시스템에 의해 고유하게 결정되지 않는 변수를 말합니다.

다음 연립선형방정식을 예로 들어 보겠습니다.

$$
\begin{matrix}
-2x_1 + 2x_2 - 2x_3 + 2x_4& = & 0\\
x_1 - 2x_2 -2x_3 \hspace{1.2cm}& = & -1\\
x_1 \hspace{1.2cm} + 2x_3 -2x_4 & = & 1
\end{matrix}
$$


In [8]:
B_augmented = np.array([[-2,2,-2,2,0],[1,-2,-2,0,-1],[1,0,2,-2,1]])
B_augmented_reduced = full_row_reduction(B_augmented)
print(B_augmented_reduced)

[[ 1.  0.  0. -2.  1.]
 [ 0.  1.  0. -1.  1.]
 [ 0.  0.  1.  0.  0.]]


RREF로 대표되는 연립선형방정식을 확인해보겠습니다.

$$
\begin{matrix}
x_1 \quad\quad\quad\quad -2x_4& = & 1\\
 x_2 \quad\quad -x_4 & = & 1\\
x_3 \quad\quad & = & 0
\end{matrix}
$$

$x_3 = 0$이며, 다른 변수들은 앞의 두 방정식을 통해 서로 연관되어 있음을 알 수 있습니다. 즉, $x_3$의 값은 시스템에 의해 고유하게 결정되지만 다른 변수의 값은 그렇지 않다는 것을 의미합니다. 해를 완성하려면 $x_1$, $x_2$ 또는 $x_4$ 중 하나의 값을 선택한 다음 처음 두 방정식을 사용하여 다른 두 변수의 값을 계산해야 합니다. 

RREF를 효율적으로 활용하기 위해 항상 자유 변수는 피벗이 포함되지 않은 배열의 열에 해당하는 변수로 선택합니다. 이 경우 네 번째 열에 피벗이 없으므로 $x_4$가 자유 변수가 됩니다. 이제 $x_4 = 2$를 선택하면 $x_1=5$와 $x_2 = 3$을 확인할 수 있습니다. 여기서 $x_4$는 *어떤 값*도 취할 수 있다는 것을 알아두는 것이 중요합니다.  그런 다음 $x_1$과 $x_2$의 값을 계산하여 시스템에 대한 해를 완성할 수 있습니다.

여기서 잠시 멈춰서 이 시스템에 대한 *가능한 모든 해*을 설명하는 방법을 생각해 볼 수 있습니다. 한 가지 아이디어는 자유 변수에 매개변수(예: $x_4 = s$)를 지정한 다음 다른 변수를 매개변수로 표현하는 것입니다.

$$
\begin{matrix}
x_1 & = & 1 + 2s\\
 x_2  & = & 1 + s\\
x_3 & = & 0 \\
x_4 & = & s
\end{matrix}
$$

중요한 점은 **선형 시스템은 하나 이상의 자유 변수를 포함할 경우 해의 개수가 무한대라는 점입니다.** RREF는 자유 변수가 있는지 여부를 한눈에 알 수 있기 때문에 유용합니다. 피벗이 없는 열이 있다면 시스템에 자유 변수가 있다는 뜻이며, 이는 시스템에 고유한 해가 없다는 것을 의미합니다.

## 피벗 위치와 불능(inconsistent)

이제 선형 시스템에 일관성이 있는지 확인하는 문제로 관심을 돌리겠습니다. 다시 한 번 구체적인 예를 살펴보고 RREF를 활용하겠습니다.

$$
\begin{matrix}
x_1 - 2x_2 + 2x_3 & = & 0\\
2x_1 + 2x_2 + 2x_3  & = & 1\\
-x_2 - x_3 & = & -2 \\
-2x_1 -x_2 - x_3 & = & 0
\end{matrix}
$$

In [9]:
D_augmented = np.array([[1,-2,2,0],[2,2,2,1],[0,-1,-1,-2],[-2,-1,-1,0]])
D_augmented_reduced = full_row_reduction(D_augmented)
print(D_augmented_reduced)

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


RREF로 표시되는 시스템을 작성하면 이 시스템에 해가 없음을 분명히 알 수 있습니다.

$$
\begin{matrix}
x_1 & = & 0\\
 x_2  & = & 0\\
x_3 & = & 0 \\
0 & = & 1
\end{matrix}
$$

이 경우 RREF를 해석할 때 주의해야 합니다. 우리는 $x_1 = x_2 = x_3 = 0$을 설정하면 세 개의 방정식은 만족하지만 한 개의 방정식은 만족하지 않는다고 결론을 내리고 싶을 수 있습니다. 하지만 원래의 시스템을 되돌아보면 그렇지 않다는 것을 알 수 있습니다. $x_1 = x_2 = x_3 = 0$을 설정하면 첫 번째와 마지막 방정식만 만족합니다.

다시 한 번 피벗 위치와 불능에 관해 결론을 연결해 보겠습니다. 특정 행의 모든 계수가 $0$이지만 오른쪽이 $0$이 아닌 경우 불일치가 발생합니다. 즉, **선형 시스템이 불능이라는 것을 의미합니다**.

## 피벗 위치와 유일 해(unique solutions)

마지막 예제에서는 시스템에 고유한 솔루션이 있는 일반적인 경우를 고려하고 RREF를 다시 고려합니다.

$$
\begin{matrix}
x_1 - 2x_2 + 2x_3 & = & 0\\
-x_2 - x_3 & = & -2 \\
-2x_1 -x_2 - x_3 & = & 0
\end{matrix}
$$


In [10]:
G_augmented = np.array([[1,-2,2,0],[0,-1,-1,-2],[-2,-1,-1,0]])
G_augmented_reduced = full_row_reduction(G_augmented)
print(G_augmented_reduced)

[[ 1.    0.    0.   -1.  ]
 [ 0.    1.    0.    0.75]
 [ 0.    0.    1.    1.25]]


RREF는 이제 고유한 솔루션을 직접 제공합니다.

$$
\begin{matrix}
x_1 & = & -1\\
 x_2  & = & 0.75\\
x_3 & = & 1.25
\end{matrix}
$$

앞의 두 가지 경우가 발생하지 않을 때 정확히 고유한 해를 갖는 시스템이 있다는 것을 인식하는 것이 중요합니다. 이를 명시적으로 설명하자면, 행렬의 **마지막 열**을 제외한 모든 열에 피벗이 있을 때 **시스템은 고유한 해**를 갖게 됩니다.  

## 피벗 위치 및 선형 시스템

이제 $m\times (n+1)$ 행렬 $[A|B]$의 피벗 위치에 따라 $AX=B$ 시스템에 대한 해집합을 요약합니다. 이러한 피벗 위치는 RREF를 계산하면 알 수 있습니다.

- 특정 행의 모든 계수가 $0$이지만 오른쪽이 $0$이 아닌 경우 불일치가 발생, 시스템에 해가 없습니다. 
- $[A|B]의 $n$ 열 중 하나라도 피벗이 *없는* 경우, 자유 변수가 존재하므로 시스템에 고유한 해가 없습니다. 
- 마지막 열을 제외한 각 열에 피벗이 있는 경우, 시스템에 고유한 해가 있습니다.

이 세 문장을 통해 주어진 시스템 $AX=B$가 고유한 해를 갖는지, 무한한 수의 해를 갖는지, 아니면 해가 전혀 없는지 확인할 수 있습니다. 선형 시스템을 분류하는 이 방법은 앞으로의 논의에서 중요한 역할을 할 것입니다.